# 用户行为分析

In [2]:
import pandas as pd
import numpy as np
import pandasql as ps
import warnings
warnings.filterwarnings('ignore')
from datetime import datetime,timedelta

## 1.数据处理

### 数据流引入

In [9]:
data = pd.read_csv('./user_action_2020-01-1to7.csv')

# 增加列名
data.columns=['user_id','page','behavior_type','sp_id','time','session']
# 转化时间格式
data['time'] = pd.to_datetime(data['time'])
# 去掉8号的数据
data = data[data['time']<'2020-01-08']

data.head()

,user_id,page,behavior_type,sp_id,time,session
0,10111119,5,提交订单,'SP_183'/'SP_708',2020-01-01 21:47:38,3f7d6f552b7d469d9a0201f6a1ca41bc
1,10111119,6,支付,'SP_183'/'SP_708',2020-01-01 21:50:48,3f7d6f552b7d469d9a0201f6a1ca41bc
2,10028611,1,浏览,NaN,2020-01-01 17:21:08,4a811658a20f4f9eba10f4e927880592
3,10028611,5,提交订单,'SP_530'/'SP_461',2020-01-01 17:21:09,4a811658a20f4f9eba10f4e927880592
4,10028611,6,支付,'SP_530'/'SP_461',2020-01-01 17:23:12,4a811658a20f4f9eba10f4e927880592


### 数据概览

In [12]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2936439 entries, 0 to 2943791
Data columns (total 6 columns):
 #   Column         Dtype         
---  ------         -----         
 0   user_id        int64         
 1   page           int64         
 2   behavior_type  object        
 3   sp_id          object        
 4   time           datetime64[ns]
 5   session        object        
dtypes: datetime64[ns](1), int64(2), object(3)
memory usage: 156.8+ MB


### 数据去重

In [10]:
data.drop_duplicates(inplace=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2936439 entries, 0 to 2943791
Data columns (total 6 columns):
 #   Column         Dtype         
---  ------         -----         
 0   user_id        int64         
 1   page           int64         
 2   behavior_type  object        
 3   sp_id          object        
 4   time           datetime64[ns]
 5   session        object        
dtypes: datetime64[ns](1), int64(2), object(3)
memory usage: 156.8+ MB


In [13]:
data.tail()

,user_id,page,behavior_type,sp_id,time,session
2943787,10110021,1,浏览,NaN,2020-01-07 18:25:12,620d87ee13b24d2d9c621b6b54925f8c
2943788,10110021,2,浏览,NaN,2020-01-07 18:25:14,620d87ee13b24d2d9c621b6b54925f8c
2943789,10110021,3,浏览,SP_671,2020-01-07 18:28:14,620d87ee13b24d2d9c621b6b54925f8c
2943790,10105223,1,浏览,NaN,2020-01-07 17:23:44,9246550dae4a43189a3cf81bc1845188
2943791,10105223,2,浏览,NaN,2020-01-07 17:23:47,9246550dae4a43189a3cf81bc1845188


## 2.数据加工

### 生产page_name字段，根据page的值范围创建映射字典

In [14]:
map_dict={
    1:'首页',
    2:'列表页',
    3:'详情页',
    4:'收藏页',
    5:'购物页',
    6:'支付页' 
}
data['page_name'] = data['page'].map(map_dict)
data.head()

,user_id,page,behavior_type,sp_id,time,session,page_name
0,10111119,5,提交订单,'SP_183'/'SP_708',2020-01-01 21:47:38,3f7d6f552b7d469d9a0201f6a1ca41bc,购物页
1,10111119,6,支付,'SP_183'/'SP_708',2020-01-01 21:50:48,3f7d6f552b7d469d9a0201f6a1ca41bc,支付页
2,10028611,1,浏览,NaN,2020-01-01 17:21:08,4a811658a20f4f9eba10f4e927880592,首页
3,10028611,5,提交订单,'SP_530'/'SP_461',2020-01-01 17:21:09,4a811658a20f4f9eba10f4e927880592,购物页
4,10028611,6,支付,'SP_530'/'SP_461',2020-01-01 17:23:12,4a811658a20f4f9eba10f4e927880592,支付页


### 将父类订单拆成子订单（将一行拆成多行）

In [41]:
data_new = data.copy()

data_new=data_new.drop('sp_id',axis=1).join(data_new['sp_id'].str.split('/',expand=True).stack().reset_index(level=1,drop=True).rename('sp_id'))

data_new['sp_id'] =[i.replace("'",'') if "'" in str(i) else i for i in data_new['sp_id']]

data_new.head()


,user_id,page,behavior_type,time,session,page_name,sp_id
0,10111119,5,提交订单,2020-01-01 21:47:38,3f7d6f552b7d469d9a0201f6a1ca41bc,购物页,SP_183
0,10111119,5,提交订单,2020-01-01 21:47:38,3f7d6f552b7d469d9a0201f6a1ca41bc,购物页,SP_708
1,10111119,6,支付,2020-01-01 21:50:48,3f7d6f552b7d469d9a0201f6a1ca41bc,支付页,SP_183
1,10111119,6,支付,2020-01-01 21:50:48,3f7d6f552b7d469d9a0201f6a1ca41bc,支付页,SP_708
2,10028611,1,浏览,2020-01-01 17:21:08,4a811658a20f4f9eba10f4e927880592,首页,NaN


In [42]:
data_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7161708 entries, 0 to 2943791
Data columns (total 7 columns):
 #   Column         Dtype         
---  ------         -----         
 0   user_id        int64         
 1   page           int64         
 2   behavior_type  object        
 3   time           datetime64[ns]
 4   session        object        
 5   page_name      object        
 6   sp_id          object        
dtypes: datetime64[ns](1), int64(2), object(4)
memory usage: 437.1+ MB


In [44]:
data_new.to_csv(r'E:\数据分析\02_开课吧-数据分析实战\数据源-student\student\user_action-加工.csv',index=False)

## 3.用户行为路径分析

### 浏览-提交订单-支付路径转化

In [53]:
'''
有多少人浏览行为后，直接提交订单，最终支付成功
首先拿出三种行为数据，用user_id合并，再去重， 计算每一步的个数
'''
# 浏览
scan = data[data['behavior_type']=='浏览'].drop_duplicates(subset='user_id')

# 提交订单
order = pd.merge(data[data['behavior_type']=='浏览'],
                 data[data['behavior_type']=='提交订单'],
                 on='user_id',how='inner').drop_duplicates(subset='user_id')

# 支付
pay = pd.merge(order,data[data['behavior_type']=='支付'],
               on='user_id',how='inner').drop_duplicates(subset='user_id')

result1 = pd.DataFrame({
    '浏览':[scan.shape[0]],
    '提交订单':[order.shape[0]],
    '支付':[pay.shape[0]]
})

result1

,浏览,提交订单,支付
0,99235,5975,5969


### 浏览-收藏-提交订单-支付路径转化

In [54]:
'''
有多少人浏览行为后，先收藏，然后提交订单，最终支付成功
首先拿出四种行为数据，用user_id合并，再去重， 计算每一步的个数
'''
# 浏览
scan = data[data['behavior_type']=='浏览'].drop_duplicates(subset='user_id')

# 收藏
collect = pd.merge(data[data['behavior_type']=='浏览'],
                 data[data['behavior_type']=='收藏'],
                 on='user_id',how='inner').drop_duplicates(subset='user_id')

# 提交订单
order = pd.merge(collect,data[data['behavior_type']=='提交订单'],
               on='user_id',how='inner').drop_duplicates(subset='user_id')

# 支付
pay = pd.merge(order,data[data['behavior_type']=='支付'],
               on='user_id',how='inner').drop_duplicates(subset='user_id')

result2 = pd.DataFrame({
    '浏览':[scan.shape[0]],
    '收藏':[collect.shape[0]],
    '提交订单':[order.shape[0]],
    '支付':[pay.shape[0]]
})

result2

,浏览,收藏,提交订单,支付
0,99235,41812,2859,2857


### 浏览-加购-提交订单-支付路径转化

In [52]:
'''
有多少人浏览行为后，先加购，然后提交订单，最终支付成功
首先拿出四种行为数据，用user_id合并，再去重， 计算每一步的个数
'''
#浏览
scan = data[data['behavior_type']=='浏览'].drop_duplicates(subset='user_id')

# 加购
add = pd.merge(data[data['behavior_type']=='浏览'],
                 data[data['behavior_type']=='加购'],
                 on='user_id',how='inner').drop_duplicates(subset='user_id')

# 提交订单
order = pd.merge(add,data[data['behavior_type']=='提交订单'],
               on='user_id',how='inner').drop_duplicates(subset='user_id')

# 支付
pay = pd.merge(order,data[data['behavior_type']=='支付'],
               on='user_id',how='inner').drop_duplicates(subset='user_id')

result3 = pd.DataFrame({
    '浏览':[scan.shape[0]],
    '加购':[add.shape[0]],
    '提交订单':[order.shape[0]],
    '支付':[pay.shape[0]]
})

result3

,浏览,加购,提交订单,支付
0,99235,77079,5066,5060
